In [1]:
import boto3
from boto3.dynamodb.conditions import Key

from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from pprint import pprint

import json
from decimal import Decimal

In [2]:
host = 'search-cloud-hw-1-43gl3ui4fy5t6aqdiv2ddgoo7a.aos.us-east-1.on.aws' # cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com
region = 'us-east-1'
service = 'aos'
auth = ('cloud', 'Cloud-hw1') 

client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)



In [5]:
# host = 'localhost'
# port = 9200
# auth = ('admin','admin')
# # Create the client with SSL/TLS and hostname verification disabled.
# client = OpenSearch(
#     hosts = [{'host': host, 'port': port}],
#     http_auth = auth,
#     http_compress = True, # enables gzip compression for request bodies
#     use_ssl = True,
#     verify_certs = False,
#     ssl_assert_hostname = False,
#     ssl_show_warn = False
# )

In [3]:
index_name = 'restaurant-index'
index_body = {
  'settings': {
    'index': {
      'number_of_shards': 4
    }
  }
}

response = client.indices.create(index_name, body=index_body)

In [4]:
response

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'restaurant-index'}

In [4]:
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
table = dynamodb.Table('yelp-restaurants')


In [5]:
cuisines = ['italian', 'chinese', 'indian', 'greek', 'mexican', 'spanish','american','japanese']
counter = 0
for cuisine in cuisines:
  
    records = table.query(
      IndexName = "cuisine-index",
      KeyConditionExpression=Key('cuisine').eq(cuisine),
    )
    actions = []

    for record in records['Items']:
        client.index(
            index = 'restaurant-index',
            body = record,
            id = str(counter)
        )
        counter += 1

In [74]:
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Decimal):
            return float(obj)
        return json.JSONEncoder.default(self, obj)

# toInsert = json.loads(json.dumps( response['Items'],  cls=JSONEncoder))

    



In [64]:
# print(" ".join(toInsert))
# client.bulk(" ".join(toInsert))
# a = toInsert.replace("[","")
# a = a.replace("]","")
# a = a.replace("}},", "}} ")
# print(a)
# client.bulk(index="restaurant-index",body=actions)

RequestError: RequestError(400, 'illegal_argument_exception', 'Malformed action/metadata line [1], expected START_OBJECT or END_OBJECT but found [VALUE_STRING]')